# Backpropagation

(예제) XOR 문제 딥러닝 모델 학습

In [25]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [49]:
# 1. XOR 데이터
X = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float)
Y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float)
X

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])

In [45]:
# 2. 모델 정의
torch.manual_seed(530)

model = nn.Sequential(
    nn.Linear(2, 2),    # 은닉층 : 입력 2 -> 출력 2 랜덤 초기화
    nn.ReLU(),          # Relu 활성화 함수
    nn.Linear(2, 1),    # 출력층(마지막) : 입력 2 -> 출력 1 랜덤 초기화
    nn.Sigmoid()        # Sigmoid 활성화 함수 (확률처럼)
)

loss_fn = nn.BCELoss()  # # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)
model

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=1, bias=True)
  (3): Sigmoid()
)

In [46]:
# 3. 학습
records = []

for epoch in range(100000):
    output = model(X)           # 예측값 계산
    loss = loss_fn(output, Y)   # 손실 계산

    optimizer.zero_grad()       # 기울기 초기화
    loss.backward()             # 역전파
    optimizer.step()            # 파라미터 업데이트

    if epoch % 10000 == 0 or epoch == 1:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        records.append({
            "epoch": epoch,
            "hidden_weight": model[0].weight.detach().numpy().copy(),
            "hidden_bias": model[0].bias.detach().numpy().copy(),
            "output_weight": model[2].weight.detach().numpy().copy(),
            "output_bias": model[2].bias.detach().numpy().copy(),
        })

df_records = pd.DataFrame([
    {
        "Epoch": r["epoch"],
        "Hidden_W": r["hidden_weight"].round(3),
        "Hidden_b": r["hidden_bias"].round(3),
        "Output_W": r["output_weight"].round(3),
        "Output_b": r["output_bias"].round(3),
    }
    for r in records
])

df_records

Epoch 0, Loss: 0.7088
Epoch 1, Loss: 0.7058
Epoch 10000, Loss: 0.0017
Epoch 20000, Loss: 0.0008
Epoch 30000, Loss: 0.0005
Epoch 40000, Loss: 0.0004
Epoch 50000, Loss: 0.0003
Epoch 60000, Loss: 0.0002
Epoch 70000, Loss: 0.0002
Epoch 80000, Loss: 0.0002
Epoch 90000, Loss: 0.0002


,Epoch,Hidden_W,Hidden_b,Output_W,Output_b
0,0,"[[-0.399, -0.457], [0.606, 0.699]]","[0.655, -0.051]","[[-0.647, 0.661]]",[-0.176]
1,1,"[[-0.407, -0.465], [0.603, 0.696]]","[0.645, -0.046]","[[-0.647, 0.655]]",[-0.178]
2,10000,"[[-2.779, -2.779], [2.779, 2.779]]","[2.779, -2.779]","[[-4.763, -4.764]]",[5.855]
3,20000,"[[-2.942, -2.942], [2.941, 2.941]]","[2.942, -2.941]","[[-5.047, -5.048]]",[6.613]
4,30000,"[[-3.03, -3.03], [3.03, 3.03]]","[3.03, -3.03]","[[-5.202, -5.203]]",[7.046]
5,40000,"[[-3.091, -3.091], [3.09, 3.09]]","[3.091, -3.09]","[[-5.307, -5.308]]",[7.349]
6,50000,"[[-3.136, -3.136], [3.136, 3.136]]","[3.136, -3.136]","[[-5.387, -5.388]]",[7.583]
7,60000,"[[-3.173, -3.173], [3.172, 3.172]]","[3.173, -3.173]","[[-5.451, -5.452]]",[7.774]
8,70000,"[[-3.204, -3.204], [3.203, 3.203]]","[3.203, -3.203]","[[-5.504, -5.505]]",[7.934]
9,80000,"[[-3.23, -3.23], [3.229, 3.229]]","[3.23, -3.229]","[[-5.55, -5.551]]",[8.073]


In [ ]:
# 4. 결과 확인
with torch.no_grad():
    output = model(X)
    predicted = torch.round(output)  # 0.5 기준 반올림 (이진 분류)
    print("\n예측 결과:")
    print(predicted)


예측 결과:
tensor([[0.],
        [1.],
        [1.],
        [0.]])


### XOR 모델 학습 과정 정리 : Epoch 0 → 1

> ### 초기 파라미터 (Epoch 0)
- layer1 : 은닉층
- layer2 (마지막) : 출력층

```
W1 = [[-0.399, -0.457],
      [ 0.606,  0.699]]
b1 = [0.655, -0.051]
W2 = [[-0.647, 0.661]]
b2 = [-0.176]
```

---

> ### 순전파 (Forward)

**1. 입력 $x$ (4개 샘플, 4 by 2)**

$$
x =
\begin{bmatrix}
0 & 0 \\
0 & 1 \\
1 & 0 \\
1 & 1
\end{bmatrix}
$$

**2. 은닉층 선형합: $z_1 = x W_1^{T} + b_1$** <br>
(참고, xW와 b가 차원이 안 맞지만 토치의 브로드캐스팅 통해 연산 가능)

$$
z_1 = 
\begin{bmatrix}
0 & 0 \\
0 & 1 \\
1 & 0 \\
1 & 1
\end{bmatrix}
\begin{bmatrix}
-0.399 & 0.606 \\
-0.457 & 0.699
\end{bmatrix}
+
\begin{bmatrix}
0.655 & -0.051
\end{bmatrix}
$$

※ $b_1$는 shape이 $(2,)$이지만 **브로드캐스팅**에 의해 각 샘플마다 더해짐

$$
z_1 =
\begin{bmatrix}
0.655 & -0.051 \\
0.198 & 0.648 \\
0.256 & 0.555 \\
-0.201 & 1.254
\end{bmatrix}
$$

**3. ReLU 적용: $h = ReLU(z_1)$**

$$
h =
\begin{bmatrix}
0.655 & 0.000 \\
0.198 & 0.648 \\
0.256 & 0.555 \\
0.000 & 1.254
\end{bmatrix}
$$


**4. 출력층 선형합: $z_2 = h W_2^{T} + b_2$**

$$
z_2 = 
\begin{bmatrix}
0.655 & 0.000 \\
0.198 & 0.648 \\
0.256 & 0.555 \\
0.000 & 1.254
\end{bmatrix}
\begin{bmatrix}
-0.647 \\
0.661
\end{bmatrix}
+
\begin{bmatrix}
-0.176
\end{bmatrix}
=
\begin{bmatrix}
-0.5998 \\
0.1242 \\
0.0252 \\
0.6529
\end{bmatrix}
$$

**5. 출력값: $y_{pred} = sigmoid(z_2)$**

$$
\hat{y} =
\begin{bmatrix}
0.3544 \\
0.5310 \\
0.5063 \\
0.6577
\end{bmatrix}
$$


**6. 정답 $y$**

$$
y =
\begin{bmatrix}
0 \\
1 \\
1 \\
0
\end{bmatrix}
$$

**Binary Cross Entropy Loss**

```
Loss = 0.705775
```

---

## 🔁 2. 역전파 (Backward)
- y와 y_hat이 같기를 궁극적으로 목표
- gradient(미분)를 활용하여 w와 b 업데이트
- gradient를 통해 ... (좀 더 수식적으로 이해 필요)


**출력층 오차: dL/dz2 = (y_pred - y) / 4**

```
[[ 0.0886],
 [-0.1172],
 [-0.1234],
 [ 0.1644]]
```

**출력층 기울기**

```
dW2 = [[0.0032, 0.0617]]
db2 = 0.0123
```

**은닉층 기울기**

```
dW1 = [[ 0.0200,  0.0190],
       [ 0.0068,  0.0078]]
db1 = [ 0.0246, -0.0126]
```

---

## 🛠️ 3. 파라미터 업데이트 (lr = 0.1)

```
W1 = [[-0.401, -0.459],
      [ 0.605,  0.698]]

b1 = [0.653, -0.050]

W2 = [[-0.647, 0.655]]
b2 = [-0.177]
```

---

## ✅ Epoch 1 결과 요약

| 파라미터 | Epoch 0                        | Epoch 1                        |
|----------|----------------------------------|----------------------------------|
| W1       | [[-0.399, -0.457], [0.606, 0.699]] | [[-0.401, -0.459], [0.605, 0.698]] |
| b1       | [0.655, -0.051]                   | [0.653, -0.050]                   |
| W2       | [[-0.647, 0.661]]                 | [[-0.647, 0.655]]                 |
| b2       | [-0.176]                          | [-0.177]                          |
